# Data Validation

## 1. Handle Missing Values

### 1. Restart / safety + imports (run first)

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, corr, isnan
import traceback
from py4j.protocol import Py4JJavaError
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd

In [26]:
try:
    spark.stop()
except Exception:
    pass

# Create SparkSession with Hive support and explicit metastore URI
spark = SparkSession.builder \
    .appName("register_flights_parquet") \
    .master("spark://spark-master:7077") \
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir", "hdfs://namenode:8020/user/hive/warehouse") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .getOrCreate()

### 2. Paths and table (change if needed)

In [27]:
parquet_path = "hdfs://namenode:8020/data/parquet/flights_2006"
table_name = "flights_2006_staged"

OUT_PATH = 'hdfs://namenode:8020/data/parquet/flights_2006_cleaned'
print('To write cleaned Parquet uncomment the write command below')

To write cleaned Parquet uncomment the write command below


### 3. Read the external table (or read parquet directly)

In [30]:
parquet_path = "hdfs://namenode:8020/data/parquet/flights_2006"

print("Checking Parquet folder contents:")
spark._jsc.hadoopConfiguration().set("fs.defaultFS", "hdfs://namenode:8020")

Checking Parquet folder contents:


In [32]:
csv_path = "hdfs://namenode:8020/data/flights/2006.csv"  # update path if your CSV is elsewhere
parquet_out = "hdfs://namenode:8020/data/parquet/flights_2006"

df_csv = spark.read.option("header","true").option("inferSchema","true").csv(csv_path)
df_csv.show(3)
df_csv.write.mode("overwrite").option("compression","snappy").parquet(parquet_out)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2006|    1|        11|        3|    743|       745|   1024|      1018|           US|      343

In [33]:
df = spark.read.option("mergeSchema","true").parquet("hdfs://namenode:8020/data/parquet/flights_2006")
df.printSchema()
df.show(5, truncate=False)

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

In [36]:
# Cell: check for Hive table existence (robust across Spark versions)
table_name = 'flights_2006_staged'

# 1) SQL check
try:
    tbls = spark.sql(f"SHOW TABLES IN default LIKE '{table_name}'")
    exists_sql = tbls.count() > 0
    print('Found via SHOW TABLES IN default LIKE:', exists_sql)
except Exception as e:
    print('SHOW TABLES failed:', e); exists_sql = False

# 2) catalog.listTables fallback
exists_catalog = False
try:
    try:
        tables = spark.catalog.listTables('default')
    except TypeError:
        tables = spark.catalog.listTables()
    names = [t.name for t in tables]
    exists_catalog = table_name in names
    print('Found via catalog.listTables:', exists_catalog)
except Exception as e:
    print('catalog.listTables failed:', e)

table_exists = exists_sql or exists_catalog
print('Final table_exists:', table_exists)

Found via SHOW TABLES IN default LIKE: False
Found via catalog.listTables: False
Final table_exists: False


In [37]:
# Cell: create EXTERNAL Hive table pointing to Parquet if missing
parquet_path = 'hdfs://namenode:8020/data/parquet/flights_2006'
table_name = 'flights_2006_staged'

if not ( 'table_exists' in globals() and table_exists ):
    print('Table not found — creating EXTERNAL table using current DataFrame schema (df must exist)')
    try:
        df
    except NameError:
        raise RuntimeError("DataFrame 'df' not found — run the loader cell first.")

    # simple mapping from Spark dtypes to Hive types
    type_map = {
        'int':'INT','bigint':'BIGINT','string':'STRING','double':'DOUBLE','float':'FLOAT',
        'boolean':'BOOLEAN','tinyint':'TINYINT','smallint':'SMALLINT','decimal':'DECIMAL'
    }

    cols = []
    for name,dtype in df.dtypes:
        hive_type = type_map.get(dtype.lower(), 'STRING')
        cols.append(f"`{name}` {hive_type}")
    cols_ddl = ',\n  '.join(cols)

    create_stmt = f"""CREATE EXTERNAL TABLE IF NOT EXISTS default.{table_name} (
  {cols_ddl}
)
STORED AS PARQUET
LOCATION '{parquet_path}'"""

    print('Running CREATE EXTERNAL TABLE...')
    try:
        spark.sql(create_stmt)
        try:
            spark.sql(f"MSCK REPAIR TABLE default.{table_name}")
        except Exception as e_rep:
            print('MSCK REPAIR TABLE returned (may be OK):', e_rep)
        try:
            spark.catalog.refreshTable(table_name)
        except Exception:
            try:
                spark.catalog.refreshTable(f'default.{table_name}')
            except Exception as e_ref:
                print('catalog.refreshTable failed (non-fatal):', e_ref)
        print('CREATE TABLE executed (or table already existed).')
    except Exception:
        import traceback; traceback.print_exc()
        raise
else:
    print('Table already exists — skipping create.')

Table not found — creating EXTERNAL table using current DataFrame schema (df must exist)
Running CREATE EXTERNAL TABLE...
MSCK REPAIR TABLE returned (may be OK): 'Operation not allowed: MSCK REPAIR TABLE only works on partitioned tables: `default`.`flights_2006_staged`;'
CREATE TABLE executed (or table already existed).


In [38]:
# Cell: verify table is readable (sample and optional count)
table_name = 'flights_2006_staged'

print('SHOW TABLES (default):')
try:
    spark.sql('SHOW TABLES IN default').show(truncate=False)
except Exception as e:
    print('SHOW TABLES failed:', e)

if spark.sql(f"SHOW TABLES IN default LIKE '{table_name}'").count() > 0:
    print('\nSelecting a small sample from the Hive table:')
    try:
        spark.sql(f"SELECT * FROM default.{table_name} LIMIT 5").show(truncate=False)
    except Exception:
        import traceback; traceback.print_exc()
    # Optional: count rows (may be slow)
    # spark.sql(f"SELECT COUNT(*) AS cnt FROM default.{table_name}").show()
else:
    print('Table not present after create attempt; check create cell output for errors.')

SHOW TABLES (default):
+--------+-------------------+-----------+
|database|tableName          |isTemporary|
+--------+-------------------+-----------+
|default |flights_2006_staged|false      |
+--------+-------------------+-----------+


Selecting a small sample from the Hive table:
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-------

In [40]:
spark.sql("SELECT COUNT(*) AS cnt FROM default.flights_2006_staged").show()

+-------+
|    cnt|
+-------+
|7141922|
+-------+



### 4. Handle missing values. (Remove Rows with null or NaN values in the target column (ArrDelay), ensuring the reliability of the target variable.)

In [45]:
try:
    df
except NameError:
    raise RuntimeError("DataFrame 'df' not found — run the loader cell first")

# Cast ArrDelay to double; non-numeric values become null
df_cast = df.withColumn('ArrDelay', F.col('ArrDelay').cast(DoubleType()))

# Counts before/after filtering
count_before = df_cast.count()
clean_df = df_cast.filter(F.col('ArrDelay').isNotNull() & (~F.isnan(F.col('ArrDelay'))))
count_after = clean_df.count()

print(f'Rows before cleaning: {count_before:,}')
print(f'Rows after cleaning: {count_after:,}')
print(f'Rows removed: {count_before - count_after:,}')

# Quick sanity: show schema and a few rows from clean_df
print('\nclean_df schema:')
clean_df.printSchema()
print('\nclean_df sample rows:')
clean_df.show(5, truncate=False)

Rows before cleaning: 7,141,922
Rows after cleaning: 7,003,802
Rows removed: 138,120

clean_df schema:
root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable 

### 2. EDA

### 1. Numeric summary for ArrDelay (central tendency & spread)

In [49]:
# compute basic aggregates (fast)
agg_row = clean_df.select(
    F.count("ArrDelay").alias("count"),
    F.mean("ArrDelay").alias("mean"),
    F.stddev("ArrDelay").alias("stddev"),
    F.min("ArrDelay").alias("min"),
    F.max("ArrDelay").alias("max")
).collect()[0]

# approximate quantiles (fast, avoids full shuffle)
q_probs = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]
q_vals = clean_df.approxQuantile("ArrDelay", q_probs, 0.01)
q_map = {f"q{int(p*100)}": v for p, v in zip(q_probs, q_vals)}

# compute IQR
iqr = q_map["q75"] - q_map["q25"]

# assemble tidy table as pandas DataFrame (safe: small)
rows = [
    ("count", int(agg_row["count"])),
    ("mean", float(agg_row["mean"]) if agg_row["mean"] is not None else None),
    ("stddev", float(agg_row["stddev"]) if agg_row["stddev"] is not None else None),
    ("min", float(agg_row["min"]) if agg_row["min"] is not None else None),
    ("max", float(agg_row["max"]) if agg_row["max"] is not None else None),
    ("q1 (25%)", q_map["q25"]),
    ("median (50%)", q_map["q50"]),
    ("q3 (75%)", q_map["q75"]),
    ("IQR (q3-q1)", iqr),
    ("q95", q_map["q95"]),
    ("q99", q_map["q99"]),
    ("q5 (5%)", q_map["q5"]),
    ("q1 (1%)", q_map["q1"])
]

# reorder for readability (preferred order)
order = ["count","mean","stddev","min","q1 (1%)","q5 (5%)","q1 (25%)","median (50%)","q3 (75%)","IQR (q3-q1)","q95","q99","max"]
# build df
df_stats = pd.DataFrame(rows, columns=["metric","value"]).set_index("metric").reindex(order)

# display nicely in notebook
from IPython.display import display
display(df_stats)

,value
metric,
count,7.003802e+06
mean,8.682840e+00
stddev,3.657647e+01
min,-5.920000e+02
q1 (1%),-5.920000e+02
q5 (5%),-1.900000e+01
q1 (25%),-9.000000e+00
median (50%),0.000000e+00
q3 (75%),1.400000e+01


### 2. Distribution / buckets for ArrDelay

In [47]:
# Bucket ArrDelay into categories and show counts & percentages
from pyspark.sql import functions as F

buckets = clean_df.withColumn("delay_bucket",
    F.when(F.col("ArrDelay") <= -15, "<= -15")
     .when((F.col("ArrDelay") > -15) & (F.col("ArrDelay") <= 0), "(-15,0]")
     .when((F.col("ArrDelay") > 0) & (F.col("ArrDelay") < 15), "(0,15)")
     .otherwise(">= 15")
)

total = buckets.count()
bucket_summary = (buckets.groupBy("delay_bucket")
                         .agg(F.count("*").alias("n"))
                         .withColumn("pct", F.round(F.col("n") / total * 100, 3))
                         .orderBy(F.desc("n")))
bucket_summary.toPandas()

,delay_bucket,n,pct
0,"(-15,0]",2957589,42.228
1,>= 15,1615537,23.067
2,"(0,15)",1581829,22.585
3,<= -15,848847,12.120


### 3. Target class balance & temporal breakdown

In [50]:
# Bucket ArrDelay into categories and show counts & percentages
from pyspark.sql import functions as F

buckets = clean_df.withColumn("delay_bucket",
    F.when(F.col("ArrDelay") <= -15, "<= -15")
     .when((F.col("ArrDelay") > -15) & (F.col("ArrDelay") <= 0), "(-15,0]")
     .when((F.col("ArrDelay") > 0) & (F.col("ArrDelay") < 15), "(0,15)")
     .otherwise(">= 15")
)

total = buckets.count()
bucket_summary = (buckets.groupBy("delay_bucket")
                         .agg(F.count("*").alias("n"))
                         .withColumn("pct", F.round(F.col("n") / total * 100, 3))
                         .orderBy(F.desc("n")))
bucket_summary.toPandas()

,delay_bucket,n,pct
0,"(-15,0]",2957589,42.228
1,>= 15,1615537,23.067
2,"(0,15)",1581829,22.585
3,<= -15,848847,12.120


### 4. Top origins/destinations & average delay

In [51]:
# Overall delay rate and breakdown by Month and DayOfWeek
from pyspark.sql import functions as F

# define delayed flag
flagged = clean_df.withColumn("is_delayed", (F.col("ArrDelay") >= 15).cast("int"))

overall = flagged.agg(
    F.count("*").alias("total"),
    F.sum("is_delayed").alias("delayed")
).withColumn("pct_delayed", F.round(F.col("delayed") / F.col("total") * 100, 3))

by_month = (flagged.groupBy("Month")
                  .agg(
                      F.count("*").alias("n"),
                      F.sum("is_delayed").alias("delayed")
                  )
                  .withColumn("pct_delayed", F.round(F.col("delayed") / F.col("n") * 100, 3))
                  .orderBy(F.desc("pct_delayed"))
            )

by_dow = (flagged.groupBy("DayOfWeek")
                .agg(F.count("*").alias("n"), F.sum("is_delayed").alias("delayed"))
                .withColumn("pct_delayed", F.round(F.col("delayed") / F.col("n") * 100, 3))
                .orderBy(F.desc("pct_delayed"))
         )

print("Overall:")
display(overall.toPandas())
print("Top months by delay rate:")
display(by_month.limit(6).toPandas())
print("Top days-of-week by delay rate:")
display(by_dow.limit(7).toPandas())

Overall:


,total,delayed,pct_delayed
0,7003802,1615537,23.067


Top months by delay rate:


,Month,n,delayed,pct_delayed
0,12,585271,157128,26.847
1,6,586418,150683,25.695
2,10,599086,153067,25.550
3,7,608631,150771,24.772
4,2,518645,118610,22.869
5,8,617379,140784,22.803


Top days-of-week by delay rate:


,DayOfWeek,n,delayed,pct_delayed
0,5,1032497,279964,27.115
1,4,1028443,267915,26.051
2,1,1027988,236539,23.010
3,7,993653,226170,22.761
4,3,1022327,225876,22.094
5,2,1012483,206593,20.405
6,6,886411,172480,19.458


### We can see that the day of the week really matters for flight delays. 
* Fridays have the most delays at around 27%, while Saturdays are much lower at about 19%. This makes DayOfWeek a simple but strong predictor: a flight on Friday is much more likely to be delayed than the same flight on Saturday. It’s definitely worth including in a model, maybe along with factors like departure time or route.

In [ ]:
spark.sparkContext.cancelAllJobs()
spark.stop()